# HIGH-DIMENSIONAL METRICS IN R

## 2. How to get started

or

In [184]:
#Pkg.rm("HDMjl")

In [59]:
#Pkg.add(url = "https://github.com/d2cml-ai/HDMjl.jl#prueba", rev = "dev")

In [7]:
# import Pkg; Pkg.add(url = "https://github.com/d2cml-ai/HDMjl.jl#prueba")

In [ ]:
import Pkg, using Pkg

In [4]:
Pkg.develop(path = "E:/causal_ml/hdm_paper/prueba/HDMjl.jl")

   Resolving package versions...


    Updating 

`C:\Users\User\.julia\environments\v1.8\Project.toml`
  [8de29b41] + HDMjl v0.0.11 `E:/causal_ml/hdm_paper/prueba/HDMjl.jl`
    Updating `C:\Users\User\.julia\environments\v1.8\Manifest.toml`


  [8de29b41] + HDMjl v0.0.11 `E:/causal_ml/hdm_paper/prueba/HDMjl.jl`


In [5]:
using CodecXz, RData, DataFrames, StatsModels, 
    Statistics, Distributions, PrettyTables, GLM, CSV, HDMjl

## 3. Prediction using Approximate Sparsity

### 3.2. A Joint Significance Test for Lasso Regression.

In [36]:
using CSV
url = "https://raw.githubusercontent.com/d2cml-ai/HDMjl.jl/prueba/data/3_2.csv"
dta = DataFrame(CSV.read(download(url), DataFrame));
n, p = size(dta);
Y = dta[:,1];
X = Matrix(dta[:,2:end]);

In [38]:
lasso_reg = rlasso(X, Y, post = false);
sum_lasso = r_summary(lasso_reg)


    Post-Lasso Estimation: false
    Total number of variables: 100
    Number of selected variables: 11
    ---
     
============ ==============
  Variable    Estimate    
============ ==============
  Intercept   0.056855
  V 1         4.77121
  V 2         4.69284
  V 3         4.76568
  V 13        -0.0453685
  V 15        -0.0467382
  V 16        -0.00499617
  V 19        -0.0922336
  V 22        -0.0272553
  V 40        -0.0105032
  V 61        0.113585
  V 100       -0.0247296
============ ==============

    ----
    Multiple R-squared: 0.9912720815874809
    Adjusted R-squared: 0.9901810917859161
    

In [39]:
url = "https://raw.githubusercontent.com/d2cml-ai/HDMjl.jl/prueba/data/3_2_2.csv"
dta = DataFrame(CSV.read(download(url), DataFrame));
n, p = size(dta);
yhat_lasso = r_predict(lasso_reg) #in-sample prediction
Ynew = dta[:,1];
Xnew = Matrix(dta[:,2:end]);
yhat_lasso_new = r_predict(lasso_reg, xnew = Xnew) #out-of-sample prediction
;

In [28]:
post_lasso_reg = rlasso(X, Y, post = true) #now use post-lasso
r_summary(post_lasso_reg)


    Post-Lasso Estimation: true
    Total number of variables: 100
    Number of selected variables: 3
    ---
     
============ ============
  Variable    Estimate  
============ ============
  Intercept   0.0341043
  V 1         4.92413
  V 2         4.85787
  V 3         4.96442
============ ============

    ----
    Multiple R-squared: 0.9906284190077158
    Adjusted R-squared: 0.990335557101707
    

In [32]:
yhat_postlasso = r_predict(post_lasso_reg) #in-sample prediction
yhat_postlasso_new = r_predict(post_lasso_reg, xnew = Xnew) #in-sample prediction
;

In [106]:
MAE = mean(eachrow(hcat(abs.(Ynew - yhat_lasso_new), abs.(Ynew - yhat_postlasso_new))))
MAE = DataFrame([[MAE[1]], [MAE[2]]], :auto)
MAE = rename!(MAE, ["lasso MAE", "Post-lasso MAE"])
pretty_table(MAE, tf = tf_simple, nosubheader = true)

============ =================
  lasso MAE   Post-lasso MAE 
============ =================
   0.879583          0.78017
============ =================


## 4. Inference on Target Regression Coefficients

### 4.1. Intuition for the Orthogonality Principle in Linear Models via Partialling Out.

In [6]:
using CSV, GLM
using DataFrames

In [7]:
url = "https://raw.githubusercontent.com/d2cml-ai/HDMjl.jl/prueba/data/4_1.csv"
dta = DataFrame(CSV.read(download(url), DataFrame));
n, p = size(dta);
y = dta[:,"y"];
d = dta[:,"d"];
x = Matrix(dta[:,3:end]);

In [96]:
full_fit = lm(hcat(ones(length(y)), Matrix(dta[:,2:end])), y);
DataFrame(Estimate = coef(full_fit)[2], Std_Error = stderror(full_fit)[2])

,Estimate,Std_Error
,Float64,Float64
1,0.978075,0.0137122


In [97]:
rY_1 = lm(hcat(ones(length(y)), Matrix(dta[:,3:end])), y);
rY = y - predict(rY_1)
rD_1 = lm(hcat(ones(length(y)), Matrix(dta[:,3:end])), d);
rD = d - predict(rD_1);

In [98]:
partial_fit_ls = lm(hcat(ones(length(y)), rD), rY)
DataFrame(Estimate = coef(partial_fit_ls)[2], Std_Error = stderror(partial_fit_ls)[2])

,Estimate,Std_Error
,Float64,Float64
1,0.978075,0.0136862


In [99]:
rY_1 = rlasso(hcat(ones(length(y)), Matrix(dta[:,3:end])), y);
rY = rY_1["residuals"]
rD_1 = rlasso(hcat(ones(length(y)), Matrix(dta[:,3:end])), d);
rD = rD_1["residuals"]
partial_fit_postlasso = lm(hcat(ones(length(y)), rD), vec(rY))
DataFrame(Estimate = coef(partial_fit_postlasso)[2], Std_Error = stderror(partial_fit_postlasso)[2])

,Estimate,Std_Error
,Float64,Float64
1,0.972739,0.0136868


In [101]:
Eff = rlassoEffect(x, y, d, method = "partialling out");
r_summary(Eff);

Estimates and significance testing of the effect of target variables
  Row   Estimate.   Std. Error   t value   Pr(>|t|) 

    1    0.972739    0.0136868   71.0715        0.0
---
Signif. codes:
0 '***' 0.001 '**' 0.01 '*' 0.05 '.' 0.1 ' ' 1


In [102]:
r_print(Eff)

Coefficients:

      1 

  0.973


In [105]:
Eff = rlassoEffect(x, y, d, method = "double selection");
r_summary(Eff);

Estimates and significance testing of the effect of target variables
  Row   Estimate.   Std. Error   t value   Pr(>|t|) 

    1    0.978075    0.0141562   69.0914        0.0
---
Signif. codes:
0 '***' 0.001 '**' 0.01 '*' 0.05 '.' 0.1 ' ' 1


### 4.2. Inference: Confidence Intervals and Significance Testing. The function rlassoEffects

In [106]:
using CSV
url = "https://raw.githubusercontent.com/d2cml-ai/HDMjl.jl/prueba/data/4_2.csv"
data = DataFrame(CSV.read(download(url), DataFrame));
n, p = size(data);
y = data[:,1];
#d = dta[:,"d"];
x = Matrix(data[:,2:end]);

In [107]:
lasso_effects = rlassoEffects(x, y, index = [1,2,3,50]);

In [109]:
r_print(lasso_effects, digits = 4)

Coefficients:

     X1          X2          X3          X50    

   2.9445      3.0413      2.9754       0.072


In [110]:
r_summary(lasso_effects);

Estimates and significance testing of the effect of target variables
        Estimate.   Std. Error    t value       Pr(>|t|) 

   X1     2.94448    0.0881468    33.4043    1.1892e-244
   X2     3.04127     0.083891    36.2527    9.0141e-288
   X3      2.9754    0.0780394     38.127   4.58085e-318
  X50   0.0719553    0.0776455   0.926716       0.354074
---
Signif. codes:
0 '***' 0.001 '**' 0.01 '*' 0.05 '.' 0.1 ' ' 1


In [111]:
r_confint(lasso_effects);

              2.5%      97.5% 

   X1      2.77171    3.11724
   X2      2.87685     3.2057
   X3      2.82245    3.12836
  X50   -0.0802271   0.224138


In [112]:
r_confint(lasso_effects, joint = true);

             2.5%      97.5% 

   X1      2.7304    3.15855
   X2     2.83944    3.24311
   X3      2.7855    3.16531
  X50   -0.113325   0.257236


### 4.3. Application: the effect of gender on wage

In [8]:
url = "https://github.com/cran/hdm/raw/master/data/cps2012.rda"
cps2012 = load(download(url))["cps2012"];
n, p = size(cps2012);
size(cps2012)

(29217, 23)

In [9]:
x_formula = @formula(lnw ~ -1 + female + female*widowed + female*divorced + female*separated + female*nevermarried +
                    female*hsd08 + female*hsd911 + female*hsg + female*cg + female*ad + female*mw + female*so + female*we + female*exp1 + female*exp2 + female*exp3)
x_dframe = ModelFrame( x_formula, cps2012)
x1 = ModelMatrix(x_dframe)
X = x1.m[:,Not(1:16)];
X = hcat(x1.m[:,1:16], X)
size(X)
fom_1 =     ["widowed", "divorced", "separated", "nevermarried", "hsd08", "hsd911", "hsg", "cg", "ad", "mw", "so",
            "we", "exp1", "exp2", "exp3"];
data = cps2012[:,fom_1];
sub_data = ones(size(data)[1])
for i in 1:size(data)[2]
    if i <= (size(data)[2] -1)
        sub_data = hcat(sub_data, Matrix(data[:, i] .* data[:, Not(1:i)]) )
    end
end
sub_data = sub_data[:,2:end]
size(sub_data)
x = hcat(X, sub_data)
size(x)
filter = var.(eachcol(x)) .!= 0
x = x[:,filter]
print(size(x))
index_gender = [1,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31];
y = cps2012.lnw;
;

(29217, 116)

In [127]:
sum(x[:,32])

15.0

In [10]:
effects_female = rlassoEffects(x, y, index = index_gender);

In [12]:
r_summary(effects_female);

Estimates and significance testing of the effect of target variables
          Estimate.   Std. Error      t value      Pr(>|t|) 

   X1     -0.154923    0.0501624     -3.08843    0.00201216
  X17      0.136095    0.0906626      1.50112      0.133325
  X18      0.136939    0.0221817      6.17353    6.6782e-10
  X19     0.0233028    0.0532118     0.437925      0.661441
  X20      0.186853    0.0199424      9.36966   7.27651e-21
  X21     0.0278103     0.120914         0.23      0.818092
  X22     -0.119335    0.0518797     -2.30023     0.0214354
  X23    -0.0128898    0.0192232    -0.670533      0.502518
  X24     0.0101386    0.0183265     0.553218      0.580114
  X25    -0.0304637    0.0218061     -1.39703      0.162405
  X26   -0.00106344    0.0191918   -0.0554112      0.955811
  X27   -0.00818334    0.0193568    -0.422763      0.672468
  X28   -0.00422613    0.0211684    -0.199643       0.84176
  X29    0.00493526   0.00780428     0.632379      0.527139
  X30     -0.159519    0.0452

In [14]:
joint_CI = r_confint(effects_female, 0.95, joint = true);
joint_CI;

              2.5%        97.5% 

   X1    -0.295562   -0.0142847
  X17    -0.136261     0.408452
  X18    0.0742004     0.199678
  X19    -0.118061     0.164666
  X20     0.128705     0.245002
  X21    -0.378365     0.433985
  X22    -0.270462    0.0317919
  X23   -0.0656411    0.0398615
  X24   -0.0421792    0.0624564
  X25   -0.0964645    0.0355371
  X26    -0.055223    0.0530961
  X27   -0.0632443    0.0468776
  X28   -0.0662182     0.057766
  X29   -0.0167011    0.0265717
  X30    -0.285341   -0.0336976
  X31    0.0166404    0.0602608


### 4.4. Application: Estimation of the treatment effect in a linear model with many confounding factors

In [15]:
using LinearAlgebra

In [16]:
using GLM
using CodecXz
using RData
using DataFrames
url = "https://github.com/cran/hdm/raw/master/data/GrowthData.rda";
GrowthData = load(download(url))["GrowthData"];
y = GrowthData[:, 1];
d = GrowthData[:, 3];
X = Matrix(GrowthData[:, Not(1, 2, 3)]);
X_1 = Matrix(GrowthData[:, Not(1, 2)]);

In [17]:
Q, R = qr(hcat(ones(length(y)), X_1))
β = pinv(hcat(ones(length(y)), X_1)) * y
# ls_effect = β;
res = y - hcat(ones(length(y)), X_1) * β;
n = size(hcat(ones(length(y)), X_1))[1]
k = size(hcat(ones(length(y)), X_1))[2]

sigma2_hat = (res' * res) / (n - k)
vcov_beta_hat = sigma2_hat .* inv(hcat(ones(length(y)), X_1)' * hcat(ones(length(y)), X_1));
se = sqrt.(diag(vcov_beta_hat))

ls_effect = DataFrame(Estimate = β, stderror = se);

In [180]:
# ls_effect = lm(hcat(ones(length(y)), X_1)[:, 1:42], y);
# coef(ls_effect)

In [18]:
lasso_effect = rlassoEffect(X, y, d, method = "partialling out");
r_summary(lasso_effect);

Estimates and significance testing of the effect of target variables
  Row    Estimate.   Std. Error    t value      Pr(>|t|) 

    1   -0.0498115    0.0139364   -3.57421   0.000351288
---
Signif. codes:
0 '***' 0.001 '**' 0.01 '*' 0.05 '.' 0.1 ' ' 1


In [19]:
doublesel_effect = rlassoEffect(X, y, d, method = "double selection");
r_summary(doublesel_effect);

Estimates and significance testing of the effect of target variables
  Row    Estimate.   Std. Error    t value     Pr(>|t|) 

    1   -0.0500059    0.0157914   -3.16666   0.00154203
---
Signif. codes:
0 '***' 0.001 '**' 0.01 '*' 0.05 '.' 0.1 ' ' 1


In [20]:
using PrettyTables
table = zeros(3,2)
table[1,:] = [round(Matrix(ls_effect)[2,1], digits = 2), round.(Matrix(ls_effect)[2,2], digits = 5)]
table[2,:] = [round(lasso_effect.coefficients, digits =2), round(lasso_effect.se, digits = 5)]
table[3,:] = [round(doublesel_effect.coefficients, digits =2), round(doublesel_effect.se, digits = 5)];
index = ["full reg via ols", "partial reg
via post-lasso ", "partial reg via double selection"]
pretty_table(hcat(index, table), show_row_number = false, header = [" ", "Estimate", "Std. Error"], tf = tf_simple, nosubheader = true)

=================================== ========== =============
                                     Estimate   Std. Error 
=================================== ========== =============
                  full reg via ols      -0.01      0.02989
      partial reg\nvia post-lasso       -0.05      0.01394
  partial reg via double selection      -0.05      0.01579
=================================== ========== =============


## 5. Instrumental Variable Estimation in a High-Dimensional Setting

### 5.2. Application: Economic Development and Institutions.

In [21]:
using RDatasets, TexTables, LinearAlgebra, GLM

In [38]:
using Statistics, StatsModels, RData
url = "https://github.com/cran/hdm/raw/master/data/AJR.rda";
AJR = load(download(url))["AJR"];
y = AJR[!,"GDP"]
d = AJR[!,"Exprop"]
z = AJR[!,"logMort"];
x_formula = @formula(GDP ~ -1 + Latitude + Latitude2 + Africa + Asia + Namer + Samer
    + Latitude*Latitude2 + Latitude*Africa + Latitude*Asia + Latitude*Namer + Latitude*Samer
    + Latitude2*Africa + Latitude2*Asia + Latitude2*Namer + Latitude2*Samer
    + Africa*Asia + Africa*Namer + Africa*Samer
    + Asia*Namer + Asia*Samer
    + Namer*Samer)
x_dframe = ModelFrame( x_formula, AJR)
x1 = ModelMatrix(x_dframe)
x = x1.m
size(x)

(64, 21)

In [39]:
AJR_Xselect  = rlassoIV(x, d, y, z, select_X=true, select_Z=false);

In [40]:
r_print(AJR_Xselect)

Coefficients:

      d1     

    0.845


In [41]:
r_summary(AJR_Xselect);

Estimates and Significance Testing of the effect of target variables in the IV regression model
         coeff.        se.   t-value      p-value 

  d1   0.845027   0.269926   3.13059   0.00174458
---
Signif. codes:
0 '***' 0.001 '**' 0.01 '*' 0.05 '.' 0.1 ' ' 1


In [42]:
r_confint(AJR_Xselect);

           2.5%     97.5% 

  d1   0.315981   1.37407


In [43]:
rY_1 = lm(hcat(ones(length(y)), Matrix(AJR[:,3:end])), y);
rY = y - predict(rY_1)
rD_1 = lm(hcat(ones(length(y)), Matrix(AJR[:,3:end])), d);
rD = d - predict(rD_1);
rZ_1 = lm(hcat(ones(length(y)), Matrix(AJR[:,3:end])), z);
rZ = z - predict(rZ_1);

In [44]:
rY_1 = lm(@formula(GDP ~ Latitude + Latitude2 + Africa + Asia + Namer + Samer + Latitude*Latitude2 + Latitude*Africa + Latitude*Asia + Latitude*Namer + Latitude*Samer
          + Latitude2*Africa + Latitude2*Asia + Latitude2*Namer + Latitude2*Samer + Africa*Asia + Africa*Namer + Africa*Samer
          + Asia*Namer + Asia*Samer + Namer*Samer), AJR)
rY = y - predict(rY_1)

rD_1 = lm(@formula(Exprop ~ Latitude + Latitude2 + Africa + Asia + Namer + Samer + Latitude*Latitude2 + Latitude*Africa + Latitude*Asia + Latitude*Namer + Latitude*Samer
          + Latitude2*Africa + Latitude2*Asia + Latitude2*Namer + Latitude2*Samer + Africa*Asia + Africa*Namer + Africa*Samer
          + Asia*Namer + Asia*Samer + Namer*Samer), AJR)
rD = d - predict(rD_1)

rZ_1 = lm(@formula(logMort ~ Latitude + Latitude2 + Africa + Asia + Namer + Samer + Latitude*Latitude2 + Latitude*Africa + Latitude*Asia + Latitude*Namer + Latitude*Samer
          + Latitude2*Africa + Latitude2*Asia + Latitude2*Namer + Latitude2*Samer + Africa*Asia + Africa*Namer + Africa*Samer
          + Asia*Namer + Asia*Samer + Namer*Samer), AJR)
rZ = z - predict(rZ_1);

In [45]:
ivfit_lm = tsls(rD, rY, rZ, nothing, intercept=false)
DataFrame(Estimate = ivfit_lm["coefficients"][1,2], Std_Error = ivfit_lm["se"])

,Estimate,Std_Error
,Float64,Float64
1,1.26721,1.73054


In [46]:
x_formula1 = @formula(GDP ~ Latitude + Latitude2 + Africa + Asia + Namer + Samer
    + Latitude*Latitude2 + Latitude*Africa + Latitude*Asia + Latitude*Namer + Latitude*Samer
    + Latitude2*Africa + Latitude2*Asia + Latitude2*Namer + Latitude2*Samer
    + Africa*Asia + Africa*Namer + Africa*Samer
    + Asia*Namer + Asia*Samer
    + Namer*Samer)
x_dframe1 = ModelFrame( x_formula, AJR)
x1_1 = ModelMatrix(x_dframe)
xx = x1.m;

In [47]:
rY_1 = rlasso(xx, y);
rY = rY_1["residuals"]
rD_1 = rlasso(xx, d);
rD = rD_1["residuals"]
rZ_1 = rlasso(xx, z);
rZ = rZ_1["residuals"]

ivfit_lasso = tsls(rD, rY, rZ)
DataFrame(Estimate = ivfit_lasso["coefficients"][1,2], Std_Error = ivfit_lasso["se"][1])

,Estimate,Std_Error
,Float64,Float64
1,0.845027,0.272094


### 5.3. Application: Impact of Eminent Domain Decisions on Economic Outcomes.

In [48]:
using Statistics, GLM
url = "https://github.com/cran/hdm/raw/master/data/EminentDomain.rda";
EminentDomain = load(download(url))["EminentDomain"];
z = EminentDomain["logGDP"]["z"];
x = EminentDomain["logGDP"]["x"];
d = EminentDomain["logGDP"]["d"];
y = EminentDomain["logGDP"]["y"];
x = x[:, (mean(x, dims = 1) .> 0.05)'];
z = z[:, (mean(z, dims = 1) .> 0.05)'];

In [50]:
ED_ols = lm(hcat(ones(length(vec(y))), hcat(d, x)), vec(y));
ED_2sls = tsls(d, y, z[:,1:2], x, intercept = false);

In [51]:
lasso_IV_Z = rlassoIV(x, d, y, z, select_X = false, select_Z = true);

In [53]:
r_summary(lasso_IV_Z);

Estimates and Significance Testing of the effect of target variables in the IV regression model
         coeff.        se.   t-value    p-value 

  d1   0.414602   0.290249   1.42843   0.153167
---
Signif. codes:
0 '***' 0.001 '**' 0.01 '*' 0.05 '.' 0.1 ' ' 1


In [54]:
r_confint(lasso_IV_Z);

            2.5%     97.5% 

  d1   -0.154276   0.98348


In [58]:
lasso_IV_XZ = rlassoIV(x, d, y, z, select_X = true, select_Z = true);
r_summary(lasso_IV_XZ);

Estimates and Significance Testing of the effect of target variables in the IV regression model
           coeff.        se.     t-value    p-value 

  d1   -0.0238347   0.128507   -0.185475   0.852857
---
Signif. codes:
0 '***' 0.001 '**' 0.01 '*' 0.05 '.' 0.1 ' ' 1


In [59]:
r_confint(lasso_IV_XZ);

            2.5%      97.5% 

  d1   -0.275703   0.228033


In [62]:
using PrettyTables
table = zeros(4,2)
table[1,:] = [coef(ED_ols)[2], stderror(ED_ols)[2]]
table[2,:] = [ED_2sls["coefficients"][1,2], ED_2sls["se"][1]]
table[3,:] = Matrix(r_summary(lasso_IV_Z)[:,2:3]);
table[4, :] = Matrix(r_summary(lasso_IV_XZ)[:, 2:3]);
index = ["ols regression", "IV estimation ", "selection on Z", "selection on X and Z"]
pretty_table(hcat(index, table), show_row_number = false, header = [" ", "Estimate", "Std. Error"], tf = tf_simple, nosubheader = true)

Estimates and Significance Testing of the effect of target variables in the IV regression model
         coeff.        se.   t-value    p-value 

  d1   0.414602   0.290249   1.42843   0.153167
---
Signif. codes:
0 '***' 0.001 '**' 0.01 '*' 0.05 '.' 0.1 ' ' 1
Estimates and Significance Testing of the effect of target variables in the IV regression model
           coeff.        se.     t-value    p-value 

  d1   -0.0238347   0.128507   -0.185475   0.852857
---
Signif. codes:
0 '***' 0.001 '**' 0.01 '*' 0.05 '.' 0.1 ' ' 1
======================= ============ =============
                           Estimate   Std. Error 
======================= ============ =============
        ols regression   0.00786473   0.00986593
        IV estimation    -0.0107097    0.0337652
        selection on Z     0.414602     0.290249
  selection on X and Z   -0.0238347     0.128507
======================= ============ =============


## 6. Inference on Treatment Effects in a High-Dimensional Setting

### 6.3. Application: 401(k) plan participation.

In [107]:
url = "https://github.com/cran/hdm/raw/master/data/pension.rda";
pension = load(download(url))["pension"];
y = pension[:, "tw"];
d = pension[:, "p401"];
z = pension[:, "e401"];
X = Matrix(pension[:, ["i2", "i3", "i4", "i5", "i6", "i7", "a2", "a3", "a4", "a5", "fsize", "hs", "smcol", "col", "marr", "twoearn", "db", "pira", "hown"]]);

In [109]:
pension_ate = rlassoATE(X, d, y);
r_summary(pension_ate);


    ------
    Post-Lasso estimation: true
    Intercept: true
    Control: 0
    Total number of variables: 19
    Number of selected variables: 9 
    ------
    
 
============ ============
  Variable    Estimate  
============ ============
  Intercept   -2.07033
  V 1         -0.237913
  V 3         0.618819
  V 4         0.846136
  V 5         1.10569
  V 6         1.34217
  V 10        -0.33151
  V 16        0.0382348
  V 17        0.620232
  V 18        0.335563
============ ============
rlassologit
    Estimation and significance tesing of the treatment effect
    Type: ATE
    Bootstrap: none
    
========== ========= ==========
  Coeff     SE        t.value 
========== ========= ==========
  10180.1   1930.68   5.2728
========== ========= ==========


In [110]:
pension_atet = rlassoATET(X, d, y);
r_summary(pension_atet);


    ------
    Post-Lasso estimation: true
    Intercept: true
    Control: 0
    Total number of variables: 19
    Number of selected variables: 6 
    ------
    
 
============ ============
  Variable    Estimate  
============ ============
  Intercept   -1.79587
  V 1         -0.608675
  V 5         0.622942
  V 6         0.839653
  V 16        0.199394
  V 17        0.643286
  V 18        0.374925
============ ============


rlassologit
    Estimation and significance tesing of the treatment effect
    Type: ATET
    Bootstrap: none
    
========== ========= ==========
  Coeff     SE        t.value 
========== ========= ==========
  12628.5   2944.43   4.28893
========== ========= ==========


In [111]:
pension_late = rlassoLATE(X, d, y, z);
r_summary(pension_late);


    ------
    Post-Lasso estimation: true
    Intercept: true
    Control: 0
    Total number of variables: 19
    Number of selected variables: 10 
    ------
    
 
============ ============
  Variable    Estimate  
============ ============
  Intercept   -1.58403
  V 1         -0.329602
  V 3         0.657641
  V 4         0.836492
  V 5         1.11528
  V 6         1.21348
  V 8         0.142622
  V 10        -0.299557
  V 16        0.0516196
  V 17        1.03219
  V 18        0.135758
============ ============
rlassologit
    Estimation and significance tesing of the treatment effect
    Type: LATE
    Bootstrap: none
    
========== ======== ==========
  Coeff     SE       t.value 
========== ======== ==========
  12992.1   2326.9   5.58344
========== ======== ==========


In [112]:
pension_latet = rlassoLATET(X, d, y, z);
r_summary(pension_latet);


    ------
    Post-Lasso estimation: true
    Intercept: true
    Control: 0
    Total number of variables: 19
    Number of selected variables: 5 
    ------
    
 
============ ============
  Variable    Estimate  
============ ============
  Intercept   -1.25636
  V 1         -0.714199
  V 5         0.677564
  V 6         0.794049
  V 16        0.212127
  V 17        1.05388
============ ============
rlassologit
    Estimation and significance tesing of the treatment effect
    Type: LATET
    Bootstrap: none
    
========== ========= ==========
  Coeff     SE        t.value 
========== ========= ==========
  15323.2   3645.28   4.20357
========== ========= ==========


In [153]:
using PrettyTables
table = zeros(4,2)
table[1,:] = round.(vec(r_summary(pension_ate)[:, 1:2]), digits = 2);
table[2,:] = round.(vec(r_summary(pension_atet)[:, 1:2]), digits = 2);
table[3,:] = round.(vec(r_summary(pension_late)[:, 1:2]), digits = 2);
table[4,:] = round.(vec(r_summary(pension_latet)[:, 1:2]), digits = 2);
index = ["ATE", "ATET ", "LATE", "LATET"];


    Estimation and significance tesing of the treatment effect
    Type: ATE
    Bootstrap: none
    
========== ========= ==========
  Coeff     SE        t.value 
========== ========= ==========
  10180.1   1930.68   5.2728
========== ========= ==========

    Estimation and significance tesing of the treatment effect
    Type: ATET
    Bootstrap: none
    
========== ========= ==========
  Coeff     SE        t.value 
========== ========= ==========
  12628.5   2944.43   4.28893
========== ========= ==========

    Estimation and significance tesing of the treatment effect
    Type: LATE
    Bootstrap: none
    
========== ======== ==========
  Coeff     SE       t.value 
========== ======== ==========
  12992.1   2326.9   5.58344
========== ======== ==========

    Estimation and significance tesing of the treatment effect
    Type: LATET
    Bootstrap: none
    
========== ========= ==========
  Coeff     SE        t.value 
========== ========= ==========
  15323.2   3645.28   4.

In [152]:
pretty_table(hcat(index, table), show_row_number = false, 
            header = [" ", "Estimate", "Std. Error"], tf = tf_simple, nosubheader = true)

======== ========== =============
          Estimate   Std. Error 
======== ========== =============
    ATE    10180.1      1930.68
  ATET     12628.5      2944.43
   LATE    12992.1       2326.9
  LATET    15323.2      3645.28
======== ========== =============


In [174]:
pension_ate = rlassoATE(X, z, y);
pension_atet = rlassoATET(X, z, y);
pension_late = rlassoLATE(X, d, y, z);
pension_latet = rlassoLATET(X, d, y, z);


    ------
    Post-Lasso estimation: true
    Intercept: true
    Control: 0
    Total number of variables: 19
    Number of selected variables: 10 
    ------
    
 
============ ============
  Variable    Estimate  
============ ============
  Intercept   -1.58403
  V 1         -0.329602
  V 3         0.657641
  V 4         0.836492
  V 5         1.11528
  V 6         1.21348
  V 8         0.142622
  V 10        -0.299557
  V 16        0.0516196
  V 17        1.03219
  V 18        0.135758
============ ============
rlassologit


    ------
    Post-Lasso estimation: true
    Intercept: true
    Control: 0
    Total number of variables: 19
    Number of selected variables: 5 
    ------
    
 
============ ============
  Variable    Estimate  
============ ============
  Intercept   -1.25636
  V 1         -0.714199
  V 5         0.677564
  V 6         0.794049
  V 16        0.212127
  V 17        1.05388
============ ============
rlassologit


    ------
    Post-Lasso estimation: true
    Intercept: true
    Control: 0
    Total number of variables: 19
    Number of selected variables: 10 
    ------
    
 
============ ============
  Variable    Estimate  
============ ============
  Intercept   -1.58403
  V 1         -0.329602
  V 3         0.657641
  V 4         0.836492
  V 5         1.11528
  V 6         1.21348
  V 8         0.142622
  V 10        -0.299557
  V 16        0.0516196
  V 17        1.03219
  V 18        0.135758
============ ============
rlassologit


    ------
    Post-Lasso estimation: true
    Intercept: true
    Control: 0
    Total number of variables: 19
    Number of selected variables: 5 
    ------
    
 
============ ============
  Variable    Estimate  
============ ============
  Intercept   -1.25636
  V 1         -0.714199
  V 5         0.677564
  V 6         0.794049
  V 16        0.212127
  V 17        1.05388
============ ============
rlassologit

In [182]:
table = zeros(4, 2)
table[1,:] = r_summary(pension_ate)[:, 1:2]
table[2,:] = r_summary(pension_atet)[:, 1:2]
table[3,:] = r_summary(pension_late)[:, 1:2]
table[4,:] = r_summary(pension_latet)[:, 1:2];


    Estimation and significance tesing of the treatment effect
    Type: ATE
    Bootstrap: none
    
========== ========= ==========
  Coeff     SE        t.value 
========== ========= ==========
  8491.99   1902.92   4.4626
========== ========= ==========

    Estimation and significance tesing of the treatment effect
    Type: ATET
    Bootstrap: none
    
========== ========= ==========
  Coeff     SE        t.value 
========== ========= ==========
  10795.3   2568.13   4.20357
========== ========= ==========

    Estimation and significance tesing of the treatment effect
    Type: LATE
    Bootstrap: none
    
========== ======== ==========
  Coeff     SE       t.value 
========== ======== ==========
  12992.1   2326.9   5.58344
========== ======== ==========

    Estimation and significance tesing of the treatment effect
    Type: LATET
    Bootstrap: none
    


========== ========= ==========
  Coeff     SE        t.value 
========== ========= ==========
  15323.2   3645.28   4.20357
========== ========= ==========


In [183]:
index = ["ATE", "ATET ", "LATE", "LATET"]
pretty_table(hcat(index, table), show_row_number = false, 
            header = [" ", "Estimate", "Std. Error"], tf = tf_simple, nosubheader = true)

======== ========== =============
          Estimate   Std. Error 
======== ========== =============
    ATE    8491.99      1902.92
  ATET     10795.3      2568.13
   LATE    12992.1       2326.9
  LATET    15323.2      3645.28
======== ========== =============


## 7. The Lasso Methods for Discovery of Significant Causes amongst Many Potential Causes, with Many Controls


In [28]:
using CSV
url = "https://raw.githubusercontent.com/d2cml-ai/HDMjl.jl/prueba/data/7_.csv"
data = DataFrame(CSV.read(download(url), DataFrame));
n, p = size(data);
p1 = 20;
X = data[:,2:end]
Y = data[:,1];

In [29]:
r_confint(rlassoEffects(X, Y, index = [1:p1;]), joint = true);

              2.5%      97.5% 

   X1      4.51533    5.21356
   X2    -0.313634    0.30429
   X3    -0.351839   0.186217
   X4    -0.253669   0.286815
   X5    -0.275994   0.275686
   X6    -0.320815   0.293574
   X7    -0.225692   0.300383
   X8   -0.0468017   0.473111
   X9    -0.185952   0.389624
  X10    -0.236704    0.26358
  X11    -0.314153   0.208903
  X12    -0.308581   0.265112
  X13    -0.173571   0.376241
  X14    -0.322822    0.38468
  X15    -0.321303   0.312452
  X16    -0.264319   0.330375
  X17    -0.178576    0.41633
  X18    -0.368883   0.046518
  X19     -0.10678   0.393157
  X20    -0.215219   0.254939
